In [ ]:
import numpy as np
import pandas as pd
import json
import requests
import re
import os
from datetime import datetime as dt
from pandas.io.json import json_normalize
import geocoder
import matplotlib.pylab as plt
import seaborn as sns
import shapefile
from sklearn import preprocessing
%matplotlib inline

df_lat_long = pd.read_csv("df_lat_long.csv")

date_t = dt.today().strftime("%Y%m%d")

#https://foursquare.com/developers/apps 申请网站
ID = 'I2E3FGJX0Z12Y0PHJ22W3WZBSUNDYWPJAQCFMSHJQBSAQJKG' //得申请
Paw = 'K13KOCRW22OVDQJBMYNBD5B4EMWBPBLU5CCDX4UWVL3V3T03' //得申请

def getNearbyPlaces(names, latitudes, longitudes, date_t, radius = 500, limit = 100,):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            ID, Paw, date_t, lat, lng, radius, limit)
        
        number_food = requests.get(url+'&section=food').json()['response']['totalResults']
        number_drink = requests.get(url+'&section=drinks').json()['response']['totalResults'] + requests.get(url+'&section=coffee').json()['response']['totalResults']
        
        number_shop = requests.get(url+'&section=shops').json()['response']['totalResults']
        number_art = requests.get(url+'&section=arts').json()['response']['totalResults']
        number_trending = requests.get(url+'&section=trending').json()['response']['totalResults']
        
        url_price_low = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&price=1'.format(
            ID, Paw, date_t, lat, lng, radius, limit)
        price_low = requests.get(url_price_low).json()['response']['totalResults']
        
        url_price_high = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&price=3,4'.format(
            ID, Paw, date_t, lat, lng, radius, limit)
        price_high = requests.get(url_price_high).json()['response']['totalResults']
        
        venues_list.append([(name, lat, lng, 
            number_food,number_drink,number_shop,number_art,number_trending,price_low,price_high)])
                            
                            
    selected_ven = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    selected_ven.columns = ['street_name', 'Latitude', 'Longitude', 'Food', 'Drink', 'Shope', 'Art', 'Trending']
    return(selected_ven)

df_venues = getNearbyPlaces(names = df_lat_long['street_name'],
                            latitudes = df_lat_long['Latitude'],
                            longitudes = df_lat_long['Longitude'],
                            date_t = dt.today().strftime("%Y%m%d"))

In [ ]:
df_venues.to_csv('dat_venues.csv')